# Using hcache_simple for Caching in Python

This tutorial provides a detailed walkthrough of the `hcache_simple` module,
which implements a lightweight caching mechanism.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import necessary modules.
import logging
import os
import time

import helpers.hcache_simple as hcacsimp
import helpers.hdbg as hdbg

In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

INFO  > cmd='/venv/lib/python3.12/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-4f3ae573-f3ef-4865-b9b0-386ca4221989.json'


In [ ]:
# Force a reload.
import importlib

importlib.reload(hcacsimp)

## Setting up caching

The `@hcsi.simple_cache` decorator enables caching for a function and supports both memory- and disk-based storage (json or pickle format).

We'll demonstrate this with a function that simulates a slow computation.

In [35]:
# cache_type="json": The cache will be stored in JSON format on disk.
# write_through=True: Any changes to the cache will be written to disk immediately.
@hcacsimp.simple_cache(cache_type="json", write_through=True)
def slow_square(x):
    """
    Simulate a slow function that computes the square of a number.

    The `@hcsi.simple_cache` decorator caches the results of this
    function to avoid recomputation for the same input.
    """
    # Simulate a time-consuming computation.
    time.sleep(2)
    return x**2

In [36]:
print(hcacsimp.cache_property_to_str("", "slow_square"))

# type_=user func_name=slow_square
# type_=system func_name=slow_square
type: json


## Demonstration: First and Subsequent Calls

Let's see how caching works:

- On the first call with a specific input, the function takes time to compute.
- On subsequent calls with the same input, the result is retrieved instantly from the cache.

In [ ]:
cache_file = hcacsimp._get_cache_file_name("slow_square")
hdbg.dassert_eq(cache_file, "cache.slow_square.json")
if os.path.exists(cache_file):
    os.remove(cache_file)

In [ ]:
# There should be no cache file yet.
!ls -l cache.slow_square.json

In [ ]:
# First call is slow: the result is computed and cached.
print("# First call (expected delay):")
result = slow_square(4)
print(f"Result: {result}")

In [ ]:
# The cache file is created and stores the content.
!cat cache.slow_square.json 

In [ ]:
# Second call is fast: the result is retrieved from the cache.
print("# Second call (retrieved from cache):")
result = slow_square(4)
print(f"Result: {result}")

In [ ]:
# Call another value -> cache miss.
result = slow_square(3)
print(f"Result: {result}")

In [ ]:
!cat cache.slow_square.json 

## Monitoring Cache Performance

The `hcache_simple` module provides utilities to track cache performance metrics,
such as the total number of calls, cache hits, and cache misses.

In [ ]:
# Enable cache performance monitoring for the function `slow_square`.
hcacsimp.enable_cache_perf("slow_square")

In [ ]:
# Retrieve and display cache performance statistics.
print("# Cache Performance Stats:")
print(hcacsimp.get_cache_perf_stats("slow_square"))

Explanation of Performance Metrics

- Total Calls (tot): The total number of times the function was invoked.
- Cache Hits (hits): The number of times the result was retrieved from the cache.
- Cache Misses (misses): The number of times the function had to compute the result due to a cache miss.
- Hit Rate: The percentage of calls where the result was retrieved from the cache.

In [ ]:
print("# First call (expected delay):")
result = slow_square(4)  # This call will be recorded as a cache miss.
print(f"Result: {result}")

print("\n# Second call (retrieved from cache):")
result = slow_square(4)  # This call will be recorded as a cache hit.
print(f"Result: {result}")

print("\n# Cache performance stats:")
print(hcacsimp.get_cache_perf_stats("slow_square"))

## Flush Cache to Disk

In [ ]:
# The following cell writes the current in‑memory cache to disk. This is useful
# if you want persistence across sessions.
print("# Flushing cache to disk for 'slow_square'...")
hcacsimp.flush_cache_to_disk("slow_square")

# The `hcsi.cache_stats_to_str` function provides a summary of the current cache
# state, including the number of items stored in memory and on disk.
print("\n# Cache stats:")
print(hcacsimp.cache_stats_to_str("slow_square"))

## Reset In‑Memory Cache

Now reset the in‑memory cache. After this, the in‑memory cache will be empty until reloaded from disk.

In [ ]:
print("# Resetting in-memory cache for 'slow_square'...")
hcacsimp.reset_mem_cache("slow_square")

print("\n# Cache stats:")
print(hcacsimp.cache_stats_to_str("slow_square"))

## Force Cache from Disk

Now we force the in‑memory cache to update from disk. This should repopulate our
cache based on the disk copy.

In [ ]:
print("# Forcing cache from disk for 'slow_square'...")
hcacsimp.force_cache_from_disk("slow_square")

print("\n# Cache stats:")
print(hcacsimp.cache_stats_to_str("slow_square"))

## Attempt to Reset Disk Cache

The `reset_disk_cache` function is currently not implemented (it contains an assertion).
We'll catch the expected error to confirm its behavior.

In [ ]:
try:
    print(
        "\nAttempting to reset disk cache for 'slow_square' (expected to fail)..."
    )
    hcacsimp.reset_disk_cache("slow_square")
except AssertionError:
    print("reset_disk_cache raised an AssertionError as expected.")

# Dynamic parameters

## force_refresh

In [ ]:
print(hcacsimp.get_cache_perf_stats("slow_square"))
hcacsimp.reset_cache_perf()
print(hcacsimp.get_cache_perf_stats("slow_square"))

In [ ]:
slow_square(4)

In [ ]:
print(hcacsimp.get_cache_perf_stats("slow_square"))

In [ ]:
# Force a recompute.
slow_square(4, force_refresh=True)

In [ ]:
print(hcacsimp.get_cache_perf_stats("slow_square"))

## abort_on_cache_miss

In [43]:
hcacsimp.reset_cache(interactive=False)

INFO  Before resetting memory cache:
None
INFO  After:
None
INFO  Before resetting disk cache:
None
WARNING Resetting disk cache
INFO  After:
None


In [44]:
# This call doesn't abort since it's not a cache miss.
slow_square(4, abort_on_cache_miss=True)

ValueError: Cache miss for key='{"args": [4], "kwargs": {}}'

In [38]:
# This call aborts since it's a cache miss.
try:
    slow_square(16, abort_on_cache_miss=True)
except ValueError as e:
    print(e)

Cache miss for key='{"args": [16], "kwargs": {}}'


In [45]:
slow_square(16, report_on_cache_miss=True)

'_cache_miss_'